In [9]:
import sys
!{sys.executable} -m pip install plotly
!{sys.executable} -m pip install statsmodels

In [5]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import statsmodels.api as sm

The data is saved as a CSV-file online so here I can just read it as it is. 

In [6]:
df = pd.read_csv("https://raw.githubusercontent.com/HaliaeetusAlbicilla/k24salary/master/kode24salary.csv")

In [7]:
df.head()

,Hva er din alder?,"Hvor mange år relevant, formell utdannelse har du?",Hvor mange års relevant arbeidserfaring har du?,Hva beskriver best din arbeidssituasjon?,I hvilket fylke jobber du?,Hva jobber du mest med?,"Hva er din grunnlønn? (årslønn før skatt, uten eventuelle bonuser eller overtidsbetaling)",Har du en bonusordning?,Alt i alt - sier du deg fornøyd med din egen lønn?
0,25-29,3,1,"in-house, startup",Oslo,"fullstack, web","300 000,00 kr",ja,Nei
1,25-29,5,2,konsulent,Trøndelag,"fullstack, web","350 000,00 kr",ja,Ja
2,30-34,5,10,konsulent,Vestland,"backend, web","350 000,00 kr",ja,Ja
3,40-44,5,18,konsulent,Rogaland,"fullstack, web","360 000,00 kr",ja,Ja
4,25-29,3,1,"in-house, privat sektor",Nordland,"backend, web","370 000,00 kr",nei,Ja
